In [1]:
#import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
data = pd.read_csv('fedex.csv')

In [3]:
# Data preprocessing

# Convert numerical columns to numeric types
numerical_columns = ['Actual_Shipment_Time', 'Planned_Shipment_Time', 'Planned_Delivery_Time', 'Planned_TimeofTravel', 'Shipment_Delay', 'Distance']
for column in numerical_columns:
    data[column] = pd.to_numeric(data[column], errors='coerce')

# Handle missing values
# For numerical columns, perform mean imputation
data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].mean())

# For categorical columns, perform mode imputation
categorical_columns = ['Carrier_Name', 'Source', 'Destination']
for column in categorical_columns:
    data[column] = data[column].fillna(data[column].mode()[0])

# Encode categorical variables
label_encoder = {}
for column in ['Carrier_Name', 'Source', 'Destination']:
  le =LabelEncoder()
  data[column] = le.fit_transform(data[column])
  label_encoder[column] = le



In [4]:
# Separate features and target
X = data.drop('Delivery_Status', axis=1)
y = data['Delivery_Status']

# Check and adjust target variable
if y.dtype != 'int':
    le = LabelEncoder()
    y = le.fit_transform(y)

# Convert target to categorical
y = to_categorical(y)

In [5]:
# Normalize numerical features
scaler = StandardScaler()
X[['Actual_Shipment_Time', 'Planned_Shipment_Time', 'Planned_Delivery_Time', 'Planned_TimeofTravel', 'Shipment_Delay', 'Distance']] = scaler.fit_transform(X[['Actual_Shipment_Time', 'Planned_Shipment_Time', 'Planned_Delivery_Time', 'Planned_TimeofTravel', 'Shipment_Delay', 'Distance']])

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
# Convert target to categorical if it's for classification
y = to_categorical(y)


In [7]:
# Build the ANN model
model = Sequential()
model.add(Dense(128, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))  # For classification, use softmax activation

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.1)

Epoch 1/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.5101 - loss: 158.2015 - val_accuracy: 0.6595 - val_loss: 2.9517
Epoch 2/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5919 - loss: 7.0566 - val_accuracy: 0.6595 - val_loss: 0.9525
Epoch 3/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6486 - loss: 2.2854 - val_accuracy: 0.6595 - val_loss: 0.8734
Epoch 4/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6539 - loss: 1.4126 - val_accuracy: 0.6595 - val_loss: 0.7824
Epoch 5/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6654 - loss: 1.0664 - val_accuracy: 0.6595 - val_loss: 0.7653
Epoch 6/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6621 - loss: 0.8885 - val_accuracy: 0.6595 - val_loss: 0.7683
Epoch 7/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6631 - loss: 0.8584 - val_accuracy: 0.6595 - val_loss: 0.7393
Epoch 8/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6601 - loss: 0.8682 - val_accuracy:

In [13]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6575 - loss: 0.7130
Loss: 0.7047, Accuracy: 0.6600


In [14]:
# Predictions (if needed)
predictions = model.predict(x_test)

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [17]:
df = pd.DataFrame(predictions)
df

0         1         2
0     0.671542  0.307048  0.021411
1     0.671542  0.307048  0.021411
2     0.671542  0.307048  0.021411
3     0.671542  0.307048  0.021411
4     0.671542  0.307048  0.021411
...        ...       ...       ...
3489  0.671542  0.307048  0.021411
3490  0.671542  0.307048  0.021411
3491  0.671542  0.307048  0.021411
3492  0.671542  0.307048  0.021411
3493  0.671542  0.307048  0.021411

[3494 rows x 3 columns]